# Linked Data

Testing with Omeka S API

## Setup

In [ ]:
import requests
import rdflib
from rdflib import Graph, plugin, URIRef, Literal
from rdflib.serializer import Serializer
from rdflib.plugin import register, Parser
from rdflib import Namespace, RDF, BNode, plugin, Variable, FOAF, SDO

In [ ]:
# create sample data to add to the graph
newData = {
    'Jane Austen' : {
        'https://schema.org/deathDate' : 1817,
        'https://schema.org/birthDate' : 1775
    },
    'J.D. Salinger' : { 
        'https://schema.org/deathDate' : 2010,
        'https://schema.org/deathPlace': 'https://en.wikipedia.org/wiki/New_York_City'
        },
    'Herman Melville' : { 
        'https://schema.org/deathDate' : 1862,
        'https://schema.org/deathPlace' : 'https://en.wikipedia.org/wiki/New_York_City'
        }
}

## Searching in Omeka S

Search for all the items in the specified set

In [ ]:
url = 'http://jajohnst.si676.si.umich.edu/omeka-s/api'

action = '/items'

parameters = {
#    property%5B0%5D%5Bjoiner%5D=and
#    property%5B0%5D%5Bproperty%5D=
#    property%5B0%5D%5Btype%5D=eq
#    property%5B0%5D%5Btext%5D=
#    resource_class_id%5B%5D=
#    resource_template_id%5B%5D=
    'item_set_id':195,
#    site_id=
#    owner_id=
#    'fulltext_search':'',
#    'submit':'Search'
}

In [ ]:
r = requests.get(url + action, params=parameters)

print(r.url)
print(r.status_code)

In [ ]:
r.json()

## Parse data with the RDFLib module

Using the `rdflib` module capabilities, parse this data.

First, create an RDF graph from it:

In [ ]:
g = Graph().parse(data=r.text, format='json-ld')

Now, look through the graph. The graph is a series of "triples",
which are subject-predicate-object tuples. These can be modified. example, after the initial look, you can remove all of those with the Omeka S namespace (`o`).
Note that RDFLib may drop or delete any orphaned subjects or objects that may not be part of a triple. 

In [ ]:
for s, p, o in g:
    print(f'{s} -> {p} -> {o} .')

### Outputting, saving, and serializing

Convert the graph to 'Turtle' format

In [ ]:
ser = g.serialize(format='turtle')

print(ser)

Save it to a file

In [ ]:
with open('item-set-graph-1.ttl', 'w') as f:
    f.write(ser)

## Parsing, Modifying, and Adding to the Graph

Now try to remove the Omeka data in order to get a closer look
at the collection specific data.

In [ ]:
# remove the omeka specific data
for triple in g:
    if 'http://omeka.org/s/vocabs/o#' in triple[1]:
        g.remove(triple)

In [ ]:
for s, p, o in g:
    print(f'{s} -> {p} -> {o}')

### Adding the "newData"

In [ ]:
# add the "newData" by looping (iterating) through the data
# and adding to the appropriate elements

# Note: this will only work if the Keys are in the data already on the site,
# so the data must be uploaded and added first

for nameKey in newData: 
    for s, p, o, in g.triples((None, FOAF.name, Literal(nameKey))):
        deathDate = newData[o.value]['https://schema.org/deathDate']
        deathPlace = newData[o.value]['https://schema.org/deathPlace']
        g.add((s, URIRef('https://schema.org/deathDate'), Literal(deathDate)))
        g.add((s, URIRef('https://schema.org/deathPlace'), URIRef(deathPlace)))

To demonstrate how the graph changed, serialize the new graph

In [ ]:
ser2 = g.serialize(format='turtle')

with open('item-set-graph-2.ttl', 'w') as f:
    f.write(ser2)